In [ ]:
from tierkreis.builder import graph, Namespace, Output, Const, Scope, Copyable
from tierkreis.pyruntime.python_runtime import VizRuntime, PyRuntime
import sys
from pathlib import Path

_TK_ROOT_DIR = Path("..")
sys.path.append(str(_TK_ROOT_DIR / "tests/"))
import test_worker.main

# Start a local tierkreis-viz instance and provide the port number
# in the url as "http://localhost:<port>"
cl = VizRuntime("http://localhost:3000", [test_worker.main.root])
# cl = PyRuntime([test_worker.main.root])

sig = await cl.get_signature()
bi = Namespace(sig)
pn = bi["python_nodes"]


delay = lambda: Const(5)

In [ ]:
@graph()
def arithmetic_graph(xvec) -> Output:
    xvec, x = bi.pop(pn.id_delay(delay(), xvec))
    _, y = bi.pop(pn.id_delay(delay(), xvec))
    return Output(
        bi.iadd(bi.fmul(pn.id_delay(delay(), x), Const(3)), pn.id_delay(delay(), y))
    )


cl.viz_graph(arithmetic_graph)

In [ ]:
arithmetic_graph = await cl.type_check_graph(arithmetic_graph)

In [ ]:
await cl.run_graph(arithmetic_graph, xvec=[7, 5])

In [ ]:
@graph()
def boxed_g() -> Output:
    a = Const(3)
    with Scope():
        d = bi.iadd(a, Const(2))
    e = bi.iadd(d, Const(1))
    return Output(value=e)


_ = await cl.type_check_graph(boxed_g)